# H3格网：网格平均面积 

&emsp;&emsp;H3 是一种可以覆盖球体的格网系统，其中既有六边形也有五边形单元格。你会发现大多数情况下你只会遇到六边形单元格，因为在每个分辨率下，***五边形的数量只有12个***，并且都被定位在海洋中。

&emsp;&emsp;在一个给定的分辨率内，单元格的大小会根据它们在地球上的位置而变化。我们将使用这个笔记本来计算六边形和五边形单元格大小的一些统计数据。

注意，代码中的 'res' 变量指的是 H3 网格系统的 '层次'。层次越高，六边形或五边形的面集越小。

In [3]:
# 导入 h3 和 tabulate 库
import h3
from tabulate import tabulate

# 定义一个函数来计算在给定层次下的六边形（排除五边形）的数量
def num_hexagons(level):
    '在一个分辨率下的*六边形*（不包括五边形）的数量'
    return h3.num_hexagons(level) - 12  # 在 4.0 版本中修复函数名

# 定义一个函数来计算地球的总面积，单位默认为平方公里
def earth_area(unit='km^2'):
    cells = h3.get_res0_indexes()
    
    return sum(h3.cell_area(c, unit) for c in cells)

# 定义一个函数计算在给定层次下所有五边形覆盖的总面积
def pentagon_area_total(level, unit='km^2'):
    '在一个分辨率下所有五边形覆盖的总面积'
    pentagons = h3.get_pentagon_indexes(level)

    return sum(h3.cell_area(c, unit) for c in pentagons)

# 定义一个函数计算在给定层次下一个五边形的平均面积（假设所有五边形的面积都相同）
def pentagon_area(level, unit='km^2'):
    '在一个分辨率下所有五边形的平均面积（假设所有五边形的面积相同）'
    return pentagon_area_total(level, unit)/12

# 定义一个函数计算在给定层次下一个六边形的平均面积
def hexagon_area_avg(level, unit='km^2'):
    A = earth_area(unit) - pentagon_area_total(level, unit)
    A = A/num_hexagons(level)
    
    return A

# 定义一个函数，对于每一个分辨率，计算并返回：
# - 层次
# - 平均六边形面积
# - 五边形面积
# - 五边形面积与六边形面积的比值
def compute_rows(unit='km^2'):
    """
    对于每个层次，返回：
    - 层次
    - 平均 *六边形* 面积
    - *五边形* 面积
    - 五边形面积和六边形面积的比率
    """
    for level in range(16):
        h = hexagon_area_avg(level, unit)
        p = pentagon_area(level, unit)

        yield level, h, p, p/h

In [4]:
# 定义浮点数格式，保留9位小数
float_fmt = '{:20,.9f}'
# 定义比例格式，保留4位小数
ratio_fmt = '{:.4f}'

# 定义一个函数，用于格式化浮点数
def fmt_float(x):
    s = float_fmt
    return s.format(x)

# 定义一个函数，用于格式化比例
def fmt_ratio(x):
    s = ratio_fmt
    return s.format(x)

# 定义一个列表，包含了所有层次的统计数据，数据被计算并格式化
fmt_stats = [
    (a, fmt_float(b), fmt_float(c), fmt_ratio(d))
    for a,b,c,d in compute_rows('km^2')
]

fmt_stats

[(0, ' 4,357,449.416078379', ' 2,562,182.162955499', '0.5880'),
 (1, '   609,788.441794133', '   328,434.586246471', '0.5386'),
 (2, '    86,801.780398997', '    44,930.898497880', '0.5176'),
 (3, '    12,393.434655088', '     6,315.472267516', '0.5096'),
 (4, '     1,770.347654491', '       896.582383141', '0.5064'),
 (5, '       252.903858182', '       127.785583023', '0.5053'),
 (6, '        36.129062164', '        18.238749548', '0.5048'),
 (7, '         5.161293360', '         2.604669397', '0.5047'),
 (8, '         0.737327598', '         0.372048038', '0.5046'),
 (9, '         0.105332513', '         0.053147195', '0.5046'),
 (10, '         0.015047502', '         0.007592318', '0.5046'),
 (11, '         0.002149643', '         0.001084609', '0.5046'),
 (12, '         0.000307092', '         0.000154944', '0.5046'),
 (13, '         0.000043870', '         0.000022135', '0.5046'),
 (14, '         0.000006267', '         0.000003162', '0.5046'),
 (15, '         0.000000895', '    

In [5]:
headers = [
    '层次',
    '平均六边形面积 (km^2)',
    '五边形面积* (km^2)',
    '比率 (五边形/六边形)'
]
out = tabulate(fmt_stats, headers=headers, tablefmt='pipe', stralign='right', disable_numparse=True)

out += '\n\n*: 在给定的层次下，所有的五边形都有相同的面积。'
print(out)

|   层次 |   平均六边形面积 (km^2) |   五边形面积* (km^2) |   比率 (五边形/六边形) |
|-------:|------------------------:|---------------------:|-----------------------:|
|      0 |     4,357,449.416078379 |  2,562,182.162955499 |                 0.5880 |
|      1 |       609,788.441794133 |    328,434.586246471 |                 0.5386 |
|      2 |        86,801.780398997 |     44,930.898497880 |                 0.5176 |
|      3 |        12,393.434655088 |      6,315.472267516 |                 0.5096 |
|      4 |         1,770.347654491 |        896.582383141 |                 0.5064 |
|      5 |           252.903858182 |        127.785583023 |                 0.5053 |
|      6 |            36.129062164 |         18.238749548 |                 0.5048 |
|      7 |             5.161293360 |          2.604669397 |                 0.5047 |
|      8 |             0.737327598 |          0.372048038 |                 0.5046 |
|      9 |             0.105332513 |          0.053147195 |                 0.5046 |
| 

In [6]:
from IPython.display import Markdown
Markdown(out)

|   层次 |   平均六边形面积 (km^2) |   五边形面积* (km^2) |   比率 (五边形/六边形) |
|-------:|------------------------:|---------------------:|-----------------------:|
|      0 |     4,357,449.416078379 |  2,562,182.162955499 |                 0.5880 |
|      1 |       609,788.441794133 |    328,434.586246471 |                 0.5386 |
|      2 |        86,801.780398997 |     44,930.898497880 |                 0.5176 |
|      3 |        12,393.434655088 |      6,315.472267516 |                 0.5096 |
|      4 |         1,770.347654491 |        896.582383141 |                 0.5064 |
|      5 |           252.903858182 |        127.785583023 |                 0.5053 |
|      6 |            36.129062164 |         18.238749548 |                 0.5048 |
|      7 |             5.161293360 |          2.604669397 |                 0.5047 |
|      8 |             0.737327598 |          0.372048038 |                 0.5046 |
|      9 |             0.105332513 |          0.053147195 |                 0.5046 |
|     10 |             0.015047502 |          0.007592318 |                 0.5046 |
|     11 |             0.002149643 |          0.001084609 |                 0.5046 |
|     12 |             0.000307092 |          0.000154944 |                 0.5046 |
|     13 |             0.000043870 |          0.000022135 |                 0.5046 |
|     14 |             0.000006267 |          0.000003162 |                 0.5046 |
|     15 |             0.000000895 |          0.000000452 |                 0.5046 |

*: 在给定的层次下，所有的五边形都有相同的面积。

# 更改面积单位：$m^2$ 

In [7]:
float_fmt = '{:20,.3f}'

def fmt_float(x):
    s = float_fmt
    return s.format(x)

fmt_stats = [
    (a, fmt_float(b), fmt_float(c))
    for a,b,c,_ in compute_rows('m^2')
]

fmt_stats

[(0, '4,357,449,416,078.392', '2,562,182,162,955.499'),
 (1, ' 609,788,441,794.134', ' 328,434,586,246.471'),
 (2, '  86,801,780,398.997', '  44,930,898,497.880'),
 (3, '  12,393,434,655.088', '   6,315,472,267.516'),
 (4, '   1,770,347,654.491', '     896,582,383.141'),
 (5, '     252,903,858.182', '     127,785,583.023'),
 (6, '      36,129,062.164', '      18,238,749.548'),
 (7, '       5,161,293.360', '       2,604,669.397'),
 (8, '         737,327.598', '         372,048.038'),
 (9, '         105,332.513', '          53,147.195'),
 (10, '          15,047.502', '           7,592.318'),
 (11, '           2,149.643', '           1,084.609'),
 (12, '             307.092', '             154.944'),
 (13, '              43.870', '              22.135'),
 (14, '               6.267', '               3.162'),
 (15, '               0.895', '               0.452')]

In [8]:
headers = [
    '层次',
    '平均六边形面积 (m^2)',
    '平均五边形面积(m^2)'
]
out = tabulate(fmt_stats, headers=headers, tablefmt='pipe', stralign='right', disable_numparse=True)

out += '\n\n*: 在给定的层次下，所有的五边形都有相同的面积。'
print(out)

|   层次 |   平均六边形面积 (m^2) |   平均五边形面积(m^2) |
|-------:|-----------------------:|----------------------:|
|      0 |  4,357,449,416,078.392 | 2,562,182,162,955.499 |
|      1 |    609,788,441,794.134 |   328,434,586,246.471 |
|      2 |     86,801,780,398.997 |    44,930,898,497.880 |
|      3 |     12,393,434,655.088 |     6,315,472,267.516 |
|      4 |      1,770,347,654.491 |       896,582,383.141 |
|      5 |        252,903,858.182 |       127,785,583.023 |
|      6 |         36,129,062.164 |        18,238,749.548 |
|      7 |          5,161,293.360 |         2,604,669.397 |
|      8 |            737,327.598 |           372,048.038 |
|      9 |            105,332.513 |            53,147.195 |
|     10 |             15,047.502 |             7,592.318 |
|     11 |              2,149.643 |             1,084.609 |
|     12 |                307.092 |               154.944 |
|     13 |                 43.870 |                22.135 |
|     14 |                  6.267 |                 3.16

In [9]:
Markdown(out)

|   层次 |   平均六边形面积 (m^2) |   平均五边形面积(m^2) |
|-------:|-----------------------:|----------------------:|
|      0 |  4,357,449,416,078.392 | 2,562,182,162,955.499 |
|      1 |    609,788,441,794.134 |   328,434,586,246.471 |
|      2 |     86,801,780,398.997 |    44,930,898,497.880 |
|      3 |     12,393,434,655.088 |     6,315,472,267.516 |
|      4 |      1,770,347,654.491 |       896,582,383.141 |
|      5 |        252,903,858.182 |       127,785,583.023 |
|      6 |         36,129,062.164 |        18,238,749.548 |
|      7 |          5,161,293.360 |         2,604,669.397 |
|      8 |            737,327.598 |           372,048.038 |
|      9 |            105,332.513 |            53,147.195 |
|     10 |             15,047.502 |             7,592.318 |
|     11 |              2,149.643 |             1,084.609 |
|     12 |                307.092 |               154.944 |
|     13 |                 43.870 |                22.135 |
|     14 |                  6.267 |                 3.162 |
|     15 |                  0.895 |                 0.452 |

*: 在给定的层次下，所有的五边形都有相同的面积。